In [1]:
import pygame
import itertools
import random
import numpy as np
import sys

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
### unit converters ###

def cm(Input):
    # converts cm to pixels
    return Input/18.0

def m(Input):
    # converts m to pixels
    return cm(Input)/100

def kmh(Input):
    # converts km/h to pixels per frame
    return Input/25.92

def sec(Input):
    # converts seconds to frames
    return Input*40

In [3]:
### Global constants ###

simulation_rate = 4
H = 900
B = 250
MAX_SPEED = kmh(50) # unit?
FOLLOWING_TIME = sec(1)
BUFFER = 20
SPAWN_PROBABILITY = 0.015
CAR_WIDTH = cm(180)
CAR_LENGTH = (20,27)
TRUCK_LENGTH = (42,50)
TRUCK_PROBABILITY = 0.1
ACCELERATION = 0.02
DECELERATION = -0.02
FPS = 30
LEFT = 1

# colors
brown = (150,140,120)
brown2 = (50, 40, 20)
grey1 = (240,240,255)
grey2 = (200,200,220)
grey3 = (160,160,180)
grey4 = (120,120,140)
grey5 = (80,80,100)
grey6 = (60,60,80)
grey7 = (40,40,60)
grey8 = (20,20,40)
grey9 = (10,10,30)
white = (240,240,245)
red = (255,100,100)
red2 = (220, 65, 65)

In [4]:
class Car():
    def __init__(self, length, path):
        self.length = length
        self.sections, self.blockades = path
        self.i = 0
        self.distance = 0
        self.speed = MAX_SPEED
        self.acceleration = 0
        self.special_priority = False
        self.color = pygame.Color(0)
        self.color.hsla = (random.randint(0,360), 75, 25)

    def section(self):
        return self.sections[self.i]
    
    def position(self):
        return self.section().position(self.distance)
    
    def angle(self):
        return self.section().angle(self.distance)
    
    def find_next_car(self):
        for car in reversed(self.section().cars):
            if car.distance > self.distance:
                return car
        for section in self.sections[self.i+1:]:
            if section.cars:
                return section.cars[-1]
            
    def update_acceleration(self):
        next_car = self.find_next_car()
        self.acceleration = ACCELERATION
        
        if self.i == 1:
            if any(b.cars for b in self.blockades) and not self.special_priority:
                space = self.sections[1].length - self.distance - self.length/2
                braking_distance = self.speed**2 / (2 * -DECELERATION)
                difference = space - braking_distance
                if 0 < difference < 10:
                    self.acceleration = DECELERATION

        if next_car:
            space = next_car.distance - self.distance - \
                    (self.length + next_car.length)/2 - BUFFER
            
            for i in range(self.i, len(self.sections)):
                if self.sections[i] == next_car.section():
                    break
                space += self.sections[i].length
            
            acceleration = 2*(space - self.speed*FOLLOWING_TIME) / FOLLOWING_TIME**2
            if acceleration < self.acceleration:
                self.acceleration = acceleration

    def update_speed(self):
        self.speed = round(max(0, min(MAX_SPEED, self.speed + self.acceleration)),2)

    def update_distance(self):
        self.distance += self.speed
    
    def update_section(self):
        section = self.section()

        if self.distance > section.length:
            self.distance -= section.length
            section.cars.remove(self)
            self.i += 1
            
            if self.i < len(self.sections):
                self.section().cars.append(self)
            else:
                return True
                

    def update(self):
        self.update_acceleration()
        self.update_speed()
        self.update_distance()
        return self.update_section()
        

In [5]:
class Section():
    def __init__(self, x, y, a, b, length):
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        self.length = length
        self.cars = []

In [6]:
class Line(Section):
    def position(self, s):
        return (self.x + self.a*s, self.y + self.b*s)

    def angle(self, s):
        return np.arctan2(-self.b, self.a)

In [7]:
class Curve(Section):
    def __init__(self, x, y, a, b, r):
        super().__init__(x, y, a, b, np.pi*r/2)
        self.r = r

    def position(self, s):
        c = self.a*np.pi/2 + self.b*s/self.r
        return (self.x+self.r*np.cos(c), self.y-self.r*np.sin(c))
    
    def angle(self, s):
        return (self.a+1)*np.pi/2 + self.b*s/self.r

In [8]:
class Intersection():
    def __init__(self, sections, paths, special_sections):
        self.sections = sections
        self.paths = paths
        self.special_sections = special_sections
        self.car_count = 0
        self.bike_count = 0
        self.cars_per_min = self.car_count/(pygame.time.get_ticks()*60000+1)
        self.bikes_per_min = 0
    
    def cars(self):
        for section in self.sections:
            for car in section.cars:
                yield car
        
    def spawn_cars(self):
        if random.random() < SPAWN_PROBABILITY:
            if random.random() < TRUCK_PROBABILITY:
                length = random.randint(*TRUCK_LENGTH)
            else:
                length = random.randint(*CAR_LENGTH)

            path = random.choice(self.paths)
            section0 = path[0][0]
            
            if not section0.cars or section0.cars[-1].distance > 80: 
                car = Car(length, path)
                section0.cars.append(car)

    def update_cars(self, t):
        if not t%60 and not any(s.cars and s.cars[0].speed for s in self.special_sections):
            cars = [s.cars[0] for s in self.special_sections if s.cars]
            if cars:
                random.choice(cars).special_priority = True
            
        for section in self.sections:
            for car in section.cars:
                if car.update():
                    self.current_time = pygame.time.get_ticks()/60000
                    self.car_count += 1
                    self.cars_per_min = self.car_count/self.current_time
                
    def update(self, t):
        self.spawn_cars()
        self.update_cars(t)

In [9]:
class Graphics():
    def __init__(self):
        self.screen = pygame.display.set_mode((H, H))
        pygame.font.init()
        self.font = pygame.font.SysFont('Helvetica', 18)

    def center(self, points):
        return np.array(points) + H/2
    
    def draw_car(self, car):
        p, l, w, a = car.position(), car.length, CAR_WIDTH, car.angle()
        corners = [[-l/2,w/2], [l/2,w/2], [l/2,-w/2], [-l/2,-w/2]]

        for c in corners:
            temp = c[0]*np.cos(a) - c[1]*np.sin(a) + p[0]
            c[1] = -c[0]*np.sin(a) - c[1]*np.cos(a) + p[1]
            c[0] = temp

        pygame.draw.polygon(self.screen, car.color, self.center(corners))
        
    def draw_stats(self, car_count):
        self.current_second = round(pygame.time.get_ticks()/1000, 1)
        text_time =  self.font.render('Seconds past: ' + str(self.current_second), False, (0, 0, 0))
        text_count = self.font.render('Number of cars: ' + str(intersection.car_count), False, (0, 0, 0))
        text_cars_per_minute = self.font.render('Cars per minute: ' + str(round(intersection.cars_per_min, 2)), False, (0, 0, 0))
        
        self.screen.blit(text_time,(20,20))
        self.screen.blit(text_count,(20,40))
        self.screen.blit(text_cars_per_minute,(20,60))

    
    def draw_sim(self, intersection):
        self.screen.fill((200, 200, 255))

        for car in intersection.cars():
            self.draw_car(car)

        self.draw_stats(intersection.car_count)
        pygame.display.flip()
        
    def draw_menu(self, design):
        
        global start_button
        global prev_button
        global next_button
        global about_button
        global quit_button
        global buttons
        
        screen = self.screen
        screen.fill((100,200,150))

        # Shapes
        start_button = pygame.draw.rect(screen, red, ( 250, 250, 400, 40))
        design_button = pygame.draw.rect(screen, red, (290, 310, 320, 40))
        prev_button = pygame.draw.rect(screen, red2, (250, 310, 40, 40))
        next_button = pygame.draw.rect(screen, red2, (610, 310, 40, 40))
        about_button = pygame.draw.rect(screen, red, (250, 370, 400, 40))
        quit_button = pygame.draw.rect(screen, red, (250, 430, 400, 40))
        buttons = [start_button, design_button, prev_button, next_button, about_button, quit_button]

        # Text
        fontObj = pygame.font.Font('freesansbold.ttf', 64)
        textSurfaceObj = fontObj.render('Traffic Flow', True, white)
        textRectObj = textSurfaceObj.get_rect()
        textRectObj.center = (450, 100)
        screen.blit(textSurfaceObj, textRectObj)

        fontObj = pygame.font.Font('freesansbold.ttf', 21)
        textSurfaceObj = fontObj.render('By Ravi & Jelle', True, white)
        textRectObj = textSurfaceObj.get_rect()
        textRectObj.center = (450, 150)
        screen.blit(textSurfaceObj, textRectObj)

        fontObj = pygame.font.Font('freesansbold.ttf', 21)
        textSurfaceObj = fontObj.render('Start', True, white)
        textRectObj = textSurfaceObj.get_rect()
        textRectObj.center = (450, 270)
        screen.blit(textSurfaceObj, textRectObj)

        fontObj = pygame.font.Font('freesansbold.ttf', 21)
        textSurfaceObj = fontObj.render('Design: ' + design.__name__.replace('_',' '), True, white)
        textRectObj = textSurfaceObj.get_rect()
        textRectObj.center = (450, 330)
        screen.blit(textSurfaceObj, textRectObj)

        fontObj = pygame.font.Font('freesansbold.ttf', 16)
        textSurfaceObj = fontObj.render('Prev', True, white)
        textRectObj = textSurfaceObj.get_rect()
        textRectObj.center = (269, 331)
        screen.blit(textSurfaceObj, textRectObj)

        fontObj = pygame.font.Font('freesansbold.ttf', 16)
        textSurfaceObj = fontObj.render('Next', True, white)
        textRectObj = textSurfaceObj.get_rect()
        textRectObj.center = (629, 331)
        screen.blit(textSurfaceObj, textRectObj)

        fontObj = pygame.font.Font('freesansbold.ttf', 21)
        textSurfaceObj = fontObj.render('About', True, white)
        textRectObj = textSurfaceObj.get_rect()
        textRectObj.center = (450, 390)
        screen.blit(textSurfaceObj, textRectObj)

        fontObj = pygame.font.Font('freesansbold.ttf', 21)
        textSurfaceObj = fontObj.render('Quit', True, white)
        textRectObj = textSurfaceObj.get_rect()
        textRectObj.center = (450, 450)
        screen.blit(textSurfaceObj, textRectObj)
        
        pygame.display.flip()

In [10]:
class Simulation():
    def __init__(self, intersection, clock, graphics):
        self.intersection = intersection
        self.clock = clock
        self.graphics = graphics
        
    def run(self):
        for t in itertools.count():
            self.clock.tick(60)
            self.intersection.update(t)
            self.graphics.draw_sim(self.intersection)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

In [11]:
class Menu():
    def __init__(self, designs, clock, graphics):
        self.designs = designs
        self.clock = clock
        self.graphics = graphics

    def run(self):
        design_index = 0

        while True:
            self.clock.tick(60)
            self.graphics.draw_menu(self.designs[design_index])
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

                elif event.type == pygame.MOUSEBUTTONDOWN and event.button == LEFT:
                    for button in buttons: 
                        if button.collidepoint(event.pos):
                            if button == start_button: 
                                return self.designs[design_index]
                            elif button == prev_button:
                                design_index = max(0, design_index-1)
                            elif button == next_button:
                                design_index = min(len(self.designs)-1, design_index+1)
                            elif button == quit_button:
                                pygame.quit()
                                sys.exit()

In [12]:
def Free_crossing():
    L = 50 # Width of track
    sections = [Line(L/2, H/2, 0, -1, H/2-L-B),
                Line(L/2, L+B, 0, -1, B),
                Line(L/2, L, 0, -1, L),
                Line(L/2, 0, 0, -1, L),
                Line(L/2, -L, 0, -1, H/2-L),
                Curve(L, L, 2, -1, L/2),
                Curve(0, 0, 0, 1, L/2)]

    for _ in range(3):
        for s in sections[-7:]:
            if isinstance(s, Line):
                sections.append(Line(s.y,-s.x,s.b,-s.a,s.length))
            else:
                sections.append(Curve(s.y,-s.x,s.a+1,s.b,s.r))

    l = [([0,1,2,3,4],[8]), ([0,1,5,25],[]), ([0,1,2,6,10,11],[8,15])]
    paths = []

    for i in range(4):
        for p, b in l:
            path_sections = [sections[(n+i*7)%28] for n in p]
            blockades = [sections[(n+i*7)%28] for n in b]
            paths.append((path_sections, blockades))
            
    special_sections = [sections[n*7+1] for n in range(4)]
            
    return sections, paths, special_sections

In [13]:
def Shark_teeth():
    L = 50 # Width of track
    sections = [Line(L/2, H/2, 0, -1, H/2-L-B),
                Line(L/2, L+B, 0, -1, B),
                Line(L/2, L, 0, -1, L),
                Line(L/2, 0, 0, -1, L),
                Line(L/2, -L, 0, -1, H/2-L),
                Curve(L, L, 2, -1, L/2),
                Curve(0, 0, 0, 1, L/2)]

    for _ in range(3):
        for s in sections[-7:]:
            if isinstance(s, Line):
                sections.append(Line(s.y,-s.x,s.b,-s.a,s.length))
            else:
                sections.append(Curve(s.y,-s.x,s.a+1,s.b,s.r))

    l = [([0,1,2,3,4],[8]), ([0,1,5,25],[]), ([0,1,2,6,10,11],[8,15])]
    paths = []

    for i in range(4):
        for p, b in l:
            path_sections = [sections[(n+i*7)%28] for n in p]
            blockades = [sections[(n+i*7)%28] for n in b]
            paths.append((path_sections, blockades))
            
    special_sections = [sections[n*7+1] for n in range(4)]
            
    return sections, paths, special_sections

In [14]:
pygame.init()
pygame.display.set_caption('Traffic Flow')
clock = pygame.time.Clock()

graphics = Graphics()
menu = Menu([Free_crossing, Shark_teeth], clock, graphics)
design = menu.run()
intersection = Intersection(*design())
simulation = Simulation(intersection, clock, graphics)
simulation.run()

SystemExit: 

/home/ravi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
